# Quran Knowledge Graph Loader

This notebook loads data from SQLite databases into a Kuzu graph database and creates relationships between nodes.

In [1]:
import kuzu
import pandas as pd
import time

# Initialize the database
db = kuzu.Database("quran_graph_db")
conn = kuzu.Connection(db)

# Install and load SQLite extension
conn.execute("INSTALL sqlite")
conn.execute("LOAD sqlite")

# Attach SQLite databases
conn.execute("ATTACH './raw_data/ayah.sqlite' as ayah (dbtype sqlite)")
conn.execute("ATTACH './raw_data/topics.sqlite' as topics (dbtype sqlite)")

## Step 1: Create Schema

First, let's create the node and relationship tables.

In [ ]:
# Create Verse node table
conn.execute("""
CREATE NODE TABLE Verse (
    id INT64,
    surah_number INT64,
    ayah_number INT64,
    verse_key STRING PRIMARY KEY,
    text STRING)
""")

# Create Topic node table
conn.execute("""
CREATE NODE TABLE Topic (
    topic_id INT64 PRIMARY KEY,
    name STRING,
    arabic_name STRING,
    parent_id INT64,
    thematic_parent_id INT64,
    ontology_parent_id INT64,
    description STRING,
    wiki_link STRING,
    thematic INT64,
    ontology INT64,
    ayahs STRING,
    related_topics STRING
)
""")

# Create HAS_TOPIC relationship table (Verse to Topic)
conn.execute("""
CREATE REL TABLE HAS_TOPIC (
    FROM Verse TO Topic
)
""")

# Create PARENT_TOPIC relationship table (Topic to Topic) with type property
conn.execute("""
CREATE REL TABLE PARENT_TOPIC (
    FROM Topic TO Topic,
    type STRING
)
""")

print("Schema created successfully.")

## Step 2: Load Data

Now, let's load the data from SQLite into Kuzu.

In [4]:
# Load verses from ayah.sqlite
conn.execute("COPY Verse FROM ayah.verses")

# Verify verses were loaded
verse_count = conn.execute("MATCH (v:Verse) RETURN count(v) AS count").get_as_df().iloc[0]['count']
print(f"Loaded {verse_count} verses into Kuzu.")

# Load topics from topics.sqlite
conn.execute("COPY Topic FROM topics.topics")

# Verify topics were loaded
topic_count = conn.execute("MATCH (t:Topic) RETURN count(t) AS count").get_as_df().iloc[0]['count']
print(f"Loaded {topic_count} topics into Kuzu.")

Loaded 6236 verses into Kuzu.
Loaded 2512 topics into Kuzu.


## Step 3: Create Topic-Topic Relationships

Now, let's create parent-child relationships between topics based on the parent_id fields.

In [5]:
# Create regular parent-child relationships
parent_result = conn.execute("""
MATCH (child:Topic), (parent:Topic)
WHERE child.parent_id = parent.topic_id
CREATE (child)-[:PARENT_TOPIC {type: 'regular'}]->(parent)
RETURN count(*) AS parent_relationships
""")
print(f"Created {parent_result.get_as_df().iloc[0]['parent_relationships']} regular parent relationships.")

# Create thematic parent relationships
thematic_result = conn.execute("""
MATCH (child:Topic), (parent:Topic)
WHERE child.thematic_parent_id = parent.topic_id
CREATE (child)-[:PARENT_TOPIC {type: 'thematic'}]->(parent)
RETURN count(*) AS thematic_parent_relationships
""")
print(f"Created {thematic_result.get_as_df().iloc[0]['thematic_parent_relationships']} thematic parent relationships.")

# Create ontology parent relationships
ontology_result = conn.execute("""
MATCH (child:Topic), (parent:Topic)
WHERE child.ontology_parent_id = parent.topic_id
CREATE (child)-[:PARENT_TOPIC {type: 'ontology'}]->(parent)
RETURN count(*) AS ontology_parent_relationships
""")
print(f"Created {ontology_result.get_as_df().iloc[0]['ontology_parent_relationships']} ontology parent relationships.")

# Count all parent-child relationships
total_result = conn.execute("MATCH ()-[r:PARENT_TOPIC]->() RETURN count(r) AS total_relationships")
print(f"Total: {total_result.get_as_df().iloc[0]['total_relationships']} parent-child relationships between topics.")

Created 732 regular parent relationships.
Created 692 thematic parent relationships.
Created 270 ontology parent relationships.
Total: 1694 parent-child relationships between topics.


## Step 4: Create Verse-Topic Relationships

Now, let's parse the ayahs field and create relationships between verses and topics.

In [6]:
# Get all topics with their ayahs field
topics_df = conn.execute("MATCH (t:Topic) RETURN t.topic_id, t.name, t.ayahs").get_as_df()

# Initialize counter for total relationships
total_relationships = 0
start_time = time.time()

# More efficient approach: process topics in batches
batch_size = 50  # Process 50 topics at a time
num_topics = len(topics_df)
num_batches = (num_topics + batch_size - 1) // batch_size  # Ceiling division

print(f"Processing {num_topics} topics in {num_batches} batches...")

for batch_idx in range(num_batches):
    start_idx = batch_idx * batch_size
    end_idx = min(start_idx + batch_size, num_topics)
    batch = topics_df.iloc[start_idx:end_idx]
    
    print(f"Processing batch {batch_idx + 1}/{num_batches} (topics {start_idx}-{end_idx-1})...")
    batch_start_time = time.time()
    
    for _, row in batch.iterrows():
        topic_id = row['t.topic_id']
        topic_name = row['t.name']
        ayahs_str = row['t.ayahs']
        
        # Skip if ayahs is empty or NaN
        if pd.isna(ayahs_str) or not ayahs_str.strip():
            continue
        
        # Split the ayahs string by comma and clean up whitespace
        verse_keys = [key.strip() for key in ayahs_str.split(',') if key.strip()]
        
        if not verse_keys:  # Skip if no valid verse keys
            continue
            
        # Create relationships for all verse keys in one query (more efficient)
        verse_keys_str = str(verse_keys).replace("'", "\"")
        
        query = f"""
        MATCH (t:Topic), (v:Verse)
        WHERE t.topic_id = {topic_id} AND v.verse_key IN {verse_keys_str}
        CREATE (v)-[:HAS_TOPIC]->(t)
        RETURN count(*) AS relationships_created
        """
        
        result = conn.execute(query)
        relationships_created = result.get_as_df().iloc[0]['relationships_created']
        total_relationships += relationships_created
    
    batch_time = time.time() - batch_start_time
    print(f"  Batch completed in {batch_time:.2f} seconds")

total_time = time.time() - start_time
print(f"Created {total_relationships} verse-topic relationships in {total_time:.2f} seconds.")

Processing 2512 topics in 51 batches...
Processing batch 1/51 (topics 0-49)...
  Batch completed in 0.09 seconds
Processing batch 2/51 (topics 50-99)...
  Batch completed in 0.05 seconds
Processing batch 3/51 (topics 100-149)...
  Batch completed in 0.06 seconds
Processing batch 4/51 (topics 150-199)...
  Batch completed in 0.05 seconds
Processing batch 5/51 (topics 200-249)...
  Batch completed in 0.06 seconds
Processing batch 6/51 (topics 250-299)...
  Batch completed in 0.06 seconds
Processing batch 7/51 (topics 300-349)...
  Batch completed in 0.05 seconds
Processing batch 8/51 (topics 350-399)...
  Batch completed in 0.05 seconds
Processing batch 9/51 (topics 400-449)...
  Batch completed in 0.06 seconds
Processing batch 10/51 (topics 450-499)...
  Batch completed in 0.15 seconds
Processing batch 11/51 (topics 500-549)...
  Batch completed in 0.12 seconds
Processing batch 12/51 (topics 550-599)...
  Batch completed in 0.08 seconds
Processing batch 13/51 (topics 600-649)...
  Batch

## Step 5: Verify the Graph

Let's verify that our graph has been properly constructed by running some example queries.

In [11]:
# Count the number of nodes and relationships
print("Graph Statistics:")
verse_count = conn.execute("MATCH (v:Verse) RETURN count(v) AS count").get_as_df().iloc[0]['count']
topic_count = conn.execute("MATCH (t:Topic) RETURN count(t) AS count").get_as_df().iloc[0]['count']
has_topic_count = conn.execute("MATCH ()-[r:HAS_TOPIC]->() RETURN count(r) AS count").get_as_df().iloc[0]['count']
parent_topic_count = conn.execute("MATCH ()-[r:PARENT_TOPIC]->() RETURN count(r) AS count").get_as_df().iloc[0]['count']

print(f"Verses: {verse_count}")
print(f"Topics: {topic_count}")
print(f"HAS_TOPIC relationships: {has_topic_count}")
print(f"PARENT_TOPIC relationships: {parent_topic_count}")

# Example query 1: Find verses related to a specific topic
print("\nExample 1: Verses about 'Patience'")
result = conn.execute("""
MATCH (v:Verse)-[:HAS_TOPIC]->(t:Topic)
WHERE t.name = 'Patience'
RETURN v.verse_key, v.text
LIMIT 5
""")
print(result.get_as_df())

# Example query 2: Find topics related to a specific verse
print("\nExample 2: Topics for verse 2:255 (Ayatul Kursi)")
result = conn.execute("""
MATCH (v:Verse)-[:HAS_TOPIC]->(t:Topic)
WHERE v.verse_key = '2:255'
RETURN t.name, t.description
""")
print(result.get_as_df())

# Example query 3: Find subtopics of a main topic
print("\nExample 3: Subtopics of 'Allah'")
result = conn.execute("""
MATCH (child:Topic)-[r:PARENT_TOPIC]->(parent:Topic)
WHERE parent.name = 'Allah'
RETURN child.name, r.type
LIMIT 10
""")
print(result.get_as_df())

# Example query 4: Find topics with different parent relationship types
print("\nExample 4: Topics with different parent relationship types")
result = conn.execute("""
MATCH (child:Topic)-[r:PARENT_TOPIC]->(parent:Topic)
RETURN r.type, count(*) AS count
""")
print(result.get_as_df())

Graph Statistics:
Verses: 6236
Topics: 2512
HAS_TOPIC relationships: 30687
PARENT_TOPIC relationships: 1694

Example 1: Verses about 'Patience'
  v.verse_key                                             v.text
0       103:1                                      وَٱلۡعَصۡرِ ١
1       103:2                   إِنَّ ٱلۡإِنسَٰنَ لَفِي خُسۡرٍ ٢
2       103:3  إِلَّا ٱلَّذِينَ ءَامَنُواْ وَعَمِلُواْ ٱلصَّٰ...
3       40:55  فَٱصۡبِرۡ إِنَّ وَعۡدَ ٱللَّهِ حَقّٞ وَٱسۡتَغۡ...
4       40:77  فَٱصۡبِرۡ إِنَّ وَعۡدَ ٱللَّهِ حَقّٞۚ فَإِمَّا...

Example 2: Topics for verse 2:255 (Ayatul Kursi)
                                      t.name  \
0                                      Allah   
1                                      Earth   
2                             Allah's Throne   
3                                  nature of   
4                                        Sky   
5                                Allah names   
6                               Intercession   
7                               

In [ ]:
conn.execute("""
MATCH (t:Topic)-[p:PARENT_TOPIC]->(n:Topic)
OPTIONAL MATCH (v:Verse)-[h:HAS_TOPIC]->(t)
RETURN t, p, n, v, h
""")